# MCP Database Connection - Demo Notebook
**Mountain Capital Partners - Ski Resort Data Analysis**

This notebook demonstrates the modular database connection package for accessing ski resort data.


## Setup
Import the necessary modules from our package.


In [ ]:
import pandas as pd
from datetime import datetime

# Import our custom modules
from db_connection import DatabaseConnection
from stored_procedures import StoredProcedures
from db_queries import execute_query_to_dataframe, map_db_tables_and_columns, get_revenue_query
from config import ResortConfig


## 1. Establish Database Connection
Connect to the MCP database using the DatabaseConnection class.


In [ ]:
# Create connection
db = DatabaseConnection()
conn = db.connect()


## 2. Using Stored Procedures
Execute the pre-defined stored procedures for Revenue, Payroll, Visits, and Weather data.

### Setup Parameters


In [ ]:
# Initialize stored procedures handler
sp = StoredProcedures(conn)

# Define parameters
database = 'Purgatory'
resort = 'Purgatory'
group_no = 46
date_ini = datetime(2025, 3, 10, 0, 0, 0)
date_end = datetime(2025, 3, 10, 23, 59, 59)


### 2.1 Revenue Data


In [ ]:
revenue_df = sp.execute_revenue(database, group_no, date_ini, date_end)
print(f"Retrieved {len(revenue_df)} revenue records")
revenue_df.head()


### 2.2 Payroll Data


In [ ]:
payroll_df = sp.execute_payroll(resort, date_ini, date_end)
print(f"Retrieved {len(payroll_df)} payroll records")
payroll_df.head()


### 2.3 Visits Data


In [ ]:
visits_df = sp.execute_visits(resort, date_ini, date_end)
print(f"Retrieved {len(visits_df)} visit records")
visits_df.head()


### 2.4 Weather Data


In [ ]:
weather_df = sp.execute_weather(resort, date_ini, date_end)
print(f"Retrieved {len(weather_df)} weather records")
weather_df.head()


## 3. Custom SQL Queries
Execute custom SQL queries and get results as DataFrames.


In [ ]:
# Example: Get recent transactions
query = """
SELECT TOP 20
    resort,
    department,
    date_time,
    pcsplit_1 as amount
FROM transact
WHERE date_time >= '2025-03-01'
ORDER BY date_time DESC
"""

df = execute_query_to_dataframe(conn, query)
print(f"Retrieved {len(df)} records")
df


## 4. Complex Revenue Query
Use the pre-built complex revenue query that aggregates across multiple profit centers.


In [ ]:
# Generate the revenue query
revenue_query = get_revenue_query(
    database='Snowbowl',
    group_no=-1,  # All groups
    date_ini='2025-03-01',
    date_end='2025-03-31 23:59:59'
)

# Execute and display results
complex_revenue_df = execute_query_to_dataframe(conn, revenue_query)
print(f"Retrieved {len(complex_revenue_df)} revenue records")

if not complex_revenue_df.empty:
    print(f"Total revenue: ${complex_revenue_df['revenue'].sum():,.2f}")
    
complex_revenue_df.head(10)


## 5. Database Structure Analysis
Explore the database schema - tables and columns.


In [ ]:
# Get all tables and columns
tables_df = map_db_tables_and_columns(conn)

print(f"Total tables: {tables_df['table'].nunique()}")
print(f"Total columns: {len(tables_df)}")

# Show column counts per table
table_summary = tables_df.groupby('table').size().sort_values(ascending=False)
print("\nTop 10 tables by column count:")
table_summary.head(10)


## 6. Available Resorts and Configuration
Display available resorts and group numbers.


In [ ]:
print("Available Resorts:")
for resort_name in ResortConfig.RESORTS:
    print(f"  • {resort_name}")

print("\nPurgatory Database Groups:")
for group_no, name in ResortConfig.PURGATORY_GROUPS.items():
    print(f"  {group_no}: {name}")

print("\nMCP Database Groups:")
for group_no, name in ResortConfig.MCP_GROUPS.items():
    print(f"  {group_no}: {name}")


## 7. Cleanup
Close the database connection when done.


In [ ]:
# Close connection
db.close()
